## 
 sieć

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


data_common = pd.read_excel("AirQualityUCI_outcome.xlsx")

features = ['PT08.S1(CO)','NMHC(GT)','C6H6(GT)','PT08.S2(NMHC)','NOx(GT)',
            'PT08.S3(NOx)','NO2(GT)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']
target = 'CO(GT)'


X_train_raw = data_common[data_common['Set']=='train'][features].values
y_train_raw = data_common[data_common['Set']=='train'][target].values.reshape(-1,1)

X_val_raw = data_common[data_common['Set']=='validation'][features].values
y_val_raw = data_common[data_common['Set']=='validation'][target].values.reshape(-1,1)

X_test_raw = data_common[data_common['Set']=='test'][features].values
y_test_raw = data_common[data_common['Set']=='test'][target].values.reshape(-1,1)


scaler_X = StandardScaler().fit(X_train_raw)
scaler_y = StandardScaler().fit(y_train_raw)

X_train, y_train = scaler_X.transform(X_train_raw), scaler_y.transform(y_train_raw)
X_val, y_val = scaler_X.transform(X_val_raw), scaler_y.transform(y_val_raw)
X_test, y_test = scaler_X.transform(X_test_raw), scaler_y.transform(y_test_raw)

print(f"Dane wspólne gotowe. Podział: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")

Dane wspólne gotowe. Podział: Train=6549, Val=1404, Test=1404


In [ ]:
import numpy as np

def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

def initialize_weights_bias(input_size, hidden_layers_sizes, output_size):
    weights = []
    biases = []
    layer_sizes = [input_size] + hidden_layers_sizes + [output_size]
    for i in range(len(layer_sizes) - 1):
        
        weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i]))
        biases.append(np.zeros((1, layer_sizes[i+1])))
    return weights, biases

def adjust_learning_rate(learning_rate, mse, previous_mse, learning_rate_adjust, threshold=1e-6):
    if mse < previous_mse:
        learning_rate *= 1.05
    else:
        learning_rate *= 0.7
    if abs(mse - previous_mse) < threshold:
        learning_rate *= learning_rate_adjust
    return learning_rate

def predict(X, weights, biases):
    output = X
    for i in range(len(weights)):
        output = np.dot(output, weights[i]) + biases[i]
        if i < len(weights) - 1:
            output = relu(output)
    return output

def train(X, y, learning_rate, learning_rate_adjust, epochs,
          hidden_layers_sizes, optimizer='gd', momentum=0.9,
          beta1=0.9, beta2=0.999, epsilon=1e-8):

    input_size = X.shape[1]
    output_size = y.shape[1]
    weights, biases = initialize_weights_bias(input_size, hidden_layers_sizes, output_size)

    
    v_w = [np.zeros_like(w) for w in weights]
    v_b = [np.zeros_like(b) for b in biases]
    m_w, v_sq_w = [np.zeros_like(w) for w in weights], [np.zeros_like(w) for w in weights]
    m_b, v_sq_b = [np.zeros_like(b) for b in biases], [np.zeros_like(b) for b in biases]

    prev_loss = np.inf

    for epoch in range(1, epochs + 1):
        
        activations = [X]
        zs = []
        for i in range(len(weights)):
            z = np.dot(activations[-1], weights[i]) + biases[i]
            zs.append(z)
            
            activations.append(z if i == len(weights)-1 else relu(z))
        
        
        error = activations[-1] - y
        loss = np.mean(error ** 2)
        
        
        learning_rate = adjust_learning_rate(learning_rate, loss, prev_loss, learning_rate_adjust)
        prev_loss = loss

        deltas = [error]
        for i in range(len(weights)-1, 0, -1):
            delta = deltas[-1].dot(weights[i].T) * relu(activations[i], derivative=True)
            deltas.append(delta)
        deltas.reverse()

        
        for i in range(len(weights)):
            grad_w = activations[i].T.dot(deltas[i]) / X.shape[0]
            grad_b = np.mean(deltas[i], axis=0, keepdims=True)

            if optimizer == 'gd':
                weights[i] -= learning_rate * grad_w
                biases[i]  -= learning_rate * grad_b
            
            elif optimizer == 'momentum':
                v_w[i] = momentum * v_w[i] - learning_rate * grad_w
                v_b[i] = momentum * v_b[i] - learning_rate * grad_b
                weights[i] += v_w[i]
                biases[i]  += v_b[i]
            
            elif optimizer == 'adam':
                m_w[i] = beta1 * m_w[i] + (1 - beta1) * grad_w
                v_sq_w[i] = beta2 * v_sq_w[i] + (1 - beta2) * (grad_w ** 2)
                m_hat = m_w[i] / (1 - beta1**epoch)
                v_hat = v_sq_w[i] / (1 - beta2**epoch)
                weights[i] -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
                
                m_b[i] = beta1 * m_b[i] + (1 - beta1) * grad_b
                v_sq_b[i] = beta2 * v_sq_b[i] + (1 - beta2) * (grad_b ** 2)
                m_hat_b = m_b[i] / (1 - beta1**epoch)
                v_hat_b = v_sq_b[i] / (1 - beta2**epoch)
                biases[i] -= learning_rate * m_hat_b / (np.sqrt(v_hat_b) + epsilon)

    return weights, biases

In [ ]:

import pandas as pd


test_params = {
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'epochs': [500, 1000, 1500, 2000],
    'layers': [[10], [10, 10], [20, 10], [32, 16, 8]],
    'momentum': [0.0, 0.5, 0.9, 0.98]
}


base = {'lr': 0.01, 'epochs': 1000, 'layers': [10, 10], 'mom': 0.9}

results_common_mlp = []

print("Rozpoczynam pełną analizę MLP dla Air Quality (4 parametry x 4 wartości x 5 powtórzeń)...")

for param_name, values in test_params.items():
    print(f"\nBadam: {param_name}")
    for val in values:
        print(f"  Wartość: {val}...", end=" ")
        
        
        curr_lr = val if param_name == 'learning_rate' else base['lr']
        curr_epochs = val if param_name == 'epochs' else base['epochs']
        curr_layers = val if param_name == 'layers' else base['layers']
        curr_mom = val if param_name == 'momentum' else base['mom']
        
        for r in range(1, 6): 
            w, b = train(X_train, y_train, learning_rate=curr_lr, learning_rate_adjust=0.0005, 
                         epochs=curr_epochs, hidden_layers_sizes=curr_layers, 
                         optimizer='momentum', momentum=curr_mom)
            
            
            for set_name, Xs, ys in [('train', X_train, y_train), ('val', X_val, y_val), ('test', X_test, y_test)]:
                preds = predict(Xs, w, b)
                
                y_real = scaler_y.inverse_transform(ys)
                p_real = scaler_y.inverse_transform(preds)
                
                results_common_mlp.append({
                    'tested_param': param_name,
                    'val': str(val),
                    'run': r,
                    'set': set_name,
                    'mse': mean_squared_error(y_real, p_real),
                    'mae': mean_absolute_error(y_real, p_real),
                    'r2': r2_score(y_real, p_real)
                })
        print("Done.")


df_common_mlp = pd.DataFrame(results_common_mlp)
df_common_mlp.to_excel('common_MLP_full_analysis.xlsx', index=False)

Rozpoczynam pełną analizę MLP dla Air Quality (4 parametry x 4 wartości x 5 powtórzeń)...

Badam: learning_rate
  Wartość: 0.001... Done.
  Wartość: 0.01... Done.
  Wartość: 0.05... Done.
  Wartość: 0.1... Done.

Badam: epochs
  Wartość: 500... Done.
  Wartość: 1000... Done.
  Wartość: 1500... Done.
  Wartość: 2000... Done.

Badam: layers
  Wartość: [10]... Done.
  Wartość: [10, 10]... Done.
  Wartość: [20, 10]... Done.
  Wartość: [32, 16, 8]... Done.

Badam: momentum
  Wartość: 0.0... Done.
  Wartość: 0.5... Done.
  Wartość: 0.9... Done.
  Wartość: 0.98... Done.


In [ ]:


def build_common_model(model_type, layers, units, lr):
    model = Sequential()
    model.add(Input(shape=(10, X_train.shape[1]))) 
    for i in range(layers):
        if model_type == 'LSTM':
            model.add(LSTM(units, return_sequences=(i < layers - 1), activation='relu'))
        else: 
            model.add(Conv1D(filters=units, kernel_size=3, padding='same', activation='relu'))
    if model_type == 'CNN': model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

def run_comprehensive_dl_common(model_type):
    results = []
    tests = {
        'layers': [1, 2, 3, 4],
        'units': [16, 32, 64, 128],
        'lr': [0.0001, 0.001, 0.01, 0.05],
        'epochs': [10, 20, 50, 80]
    }
    base = {'layers': 1, 'units': 32, 'lr': 0.001, 'epochs': 20}

    for p_name, p_values in tests.items():
        print(f"\nBadanie {model_type} - Parametr: {p_name}")
        for val in p_values:
            print(f"  Wartość {val}...", end=" ")
            for r in range(1, 6):
                cfg = base.copy()
                cfg[p_name] = val
                
                model = build_common_model(model_type, cfg['layers'], cfg['units'], cfg['lr'])
                model.fit(X_train_seq, y_train_seq, epochs=cfg['epochs'], batch_size=32, verbose=0)
                
                
                pred = model.predict(X_test_seq, verbose=0)
                p_real = scaler_y.inverse_transform(pred)
                y_real = scaler_y.inverse_transform(y_test_seq)
                
                results.append({
                    'model': model_type, 'tested_param': p_name, 'val': str(val), 'run': r,
                    'mse': mean_squared_error(y_real, p_real),
                    'mae': mean_absolute_error(y_real, p_real),
                    'r2': r2_score(y_real, p_real)
                })
            print("Done.")
    return results


res_lstm_common = run_comprehensive_dl_common('LSTM')
res_cnn_common = run_comprehensive_dl_common('CNN')


pd.concat([pd.DataFrame(res_lstm_common), pd.DataFrame(res_cnn_common)]).to_excel('common_Advanced_full_analysis.xlsx', index=False)


Badanie LSTM - Parametr: layers
  Wartość 1... Done.
  Wartość 2... Done.
  Wartość 3... Done.
  Wartość 4... Done.

Badanie LSTM - Parametr: units
  Wartość 16... Done.
  Wartość 32... Done.
  Wartość 64... Done.
  Wartość 128... Done.

Badanie LSTM - Parametr: lr
  Wartość 0.0001... Done.
  Wartość 0.001... Done.
  Wartość 0.01... Done.
  Wartość 0.05... Done.

Badanie LSTM - Parametr: epochs
  Wartość 10... Done.
  Wartość 20... Done.
  Wartość 50... Done.
  Wartość 80... Done.

Badanie CNN - Parametr: layers
  Wartość 1... Done.
  Wartość 2... Done.
  Wartość 3... Done.
  Wartość 4... Done.

Badanie CNN - Parametr: units
  Wartość 16... Done.
  Wartość 32... Done.
  Wartość 64... Done.
  Wartość 128... Done.

Badanie CNN - Parametr: lr
  Wartość 0.0001... Done.
  Wartość 0.001... Done.
  Wartość 0.01... Done.
  Wartość 0.05... Done.

Badanie CNN - Parametr: epochs
  Wartość 10... Done.
  Wartość 20... Done.
  Wartość 50... Done.
  Wartość 80... Done.
